In [ ]:
%pylab inline
import pandas as pd
from scipy import linalg
from itertools import combinations
import scipy
import scipy.io as io
import random



$\def\m#1{\mathbf{#1}}$
$\def\mm#1{\boldsymbol{#1}}$
$\def\mb#1{\mathbb{#1}}$
$\def\c#1{\mathcal{#1}}$
$\def\mr#1{\mathrm{#1}}$
$\newenvironment{rmat}{\left[\begin{array}{rrrrrrrrrrrrr}}{\end{array}\right]}$
$\newcommand\brm{\begin{rmat}}$
$\newcommand\erm{\end{rmat}}$
$\newenvironment{cmat}{\left[\begin{array}{ccccccccc}}{\end{array}\right]}$
$\newcommand\bcm{\begin{cmat}}$
$\newcommand\ecm{\end{cmat}}$

# Homework 1
## Homework guideline
- The deadline is Sep 8th 10:30am. Submission after the deadline will not be graded.

- Submit your work(your reasoning and your code) as a SINGLE .ipynb document. Please rename the document as _HW1_YOURNAME.ipynb_ (for example, _HW1_FELIX.ipynb_). You are responsible for checking that you have correctly submitted the correct document. If your code cannot run, you may receive NO point.

- Please justify all short answers with a brief explanation. If you use latex command in the markdown, **2 points** bonus will be awarded.   

- You only use the Python packages included in the following cell. You are not allowed to use other advanced package or modules unless you are permitted to.

- In your final submission include the plots produced by the unedited code as presented below, as well as any additional plots produced after editing the code during the course of a problem. You may find it necessary to copy/paste relevant code into additional cells to accomplish this.

- Feel free to use the lecture notes and other resources. But you
must understand, write, and hand in your own answers. In addition, you must write and submit
your own code in the programming part of the assignment (we may run your code).
If you copy someone else homework solution, both of you may receive ZERO point.


- Colab is preferred. However, if you use Anaconda, please download the data file locally and save it to the same folder as this homework file.

# Q1: Unfair coin (20pt)

You are given 1000 coins. Among them, 1 coin has heads on both sides. The other 999
coins are fair coins. You randomly choose a coin and toss it 10 times. Each time, the
coin turns up heads.

---


## Q1.1: Theory (10pt)
What is the probability that the coin you choose is the unfair one?

# Your Solution:
The probability we are solving for is the conditional probability that we pick the unfair coin given we tossed a coin 10 times
and flipped heads every time. To do this, we will use Bayes' Theorem:

\begin{align*}
P( \text{ unfair coin picked } | \text{ 10 heads in a row } ) &= \frac{ P( \text{ 10 heads in a row } | \text{ unfair coin picked } )*P( \text{ unfair coin picked } )}{\Sigma^{1000}_{i=1} P( \text{ 10 heads in a row } | \text{ } i\text{th coin picked } )*P( \text{ } i\text{th coin picked })}\\ 
&= \frac{1*\frac{1}{1000}}{(\frac{999}{1000}*.5^{10})+(1*\frac{1}{1000})}\\
&= \frac{\frac{1}{1000}}{\frac{1}{1000}*((999*.5^{10})+1)}\\
&= \frac{1}{1.9756}\\
&= .506
\end{align*}





---

## Q1.2: Design (10pt)
Design a Monte Carlo Simulation using numpy and scipy to verify your solution.

In [4]:
import random

#function that flips a coin, returning a 1 on Head, 0 on a Tail
def coin_flips(coin):
    outcome = coin[random.randint(0,1)]
    if outcome == "H":
        return 1
    else:
        return 0

fair_coin = ["H", "T"]
unfair_coin = ["H", "H"]

total_sims = 10000
#number of runs through the simulation, best results occur past 5000 simulations (~15 sec runtime per 1000 simulations on my machine)
heads_in_a_row = 10
#the number of tosses we wish to do, since we only want to flip up until this number anyways
number_of_fair_coins = 999
#the number of fair coins we wish to use in our simulations
iterator = 0
#iterates through the "master while loop"
total_probability = 0
#will add the probabilities of each simulation into one variable, later dividiing by the total number of simulations

while iterator < total_sims:
    #start of simulation
    iterator += 1

    sim_size = 10000
    #number of coins checked per run, keeping this at 10000 to ensure accuracy of each simulation
    total_successes = 0
    #variable keeps track of the number of successes gathered from any one simulation
    unfair_coin_count = 0
    fair_coin_count = 0
    #counts the coin type on successes

    while sim_size > 0:
        #start of one simulation through
        sim_size -= 1
        value = random.randint(0,number_of_fair_coins)
        #gathering a random coin from 1000 coins, where 0 is the unfair_coin, and anything else is a fair coin

        #if statement checks the coin. If the coin is unfair, move to next coin select. Otherwise, move to flips.
        if value == 0:
            unfair_coin_count += 1
            total_successes += 1
        else:
            toss_number = 0
            #iterative variable

            while toss_number < heads_in_a_row:
                #start of flips
                toss_number += 1

                if coin_flips(fair_coin) == 0:
                    #i.e. if we roll tails, move on by arbitrarily increasing the iterator
                    toss_number += heads_in_a_row
                elif toss_number == heads_in_a_row:
                    #if we get here with toss_number = heads_in_a_row, we know that we rolled 10 heads in a row
                    total_successes += 1
                    fair_coin_count += 1
                else:
                    continue


    #gathers the probabilities from each simulation for a final division later
    total_probability += unfair_coin_count/total_successes

#final division for the desired probability 
probability = total_probability/total_sims

print("Monte Carlo Simulation of the above inquiry estimates the probability to be:" , probability)

Monte Carlo Simulation of the above probability estimates the answer to be: 0.5053823933089252





---


---

# Q2: Dice Order (20pt)
We throw 3 dice one by one.


---

## Q2.1: Theory (10pt)
What is the probability that we obtain 3 points in strictly increasing order?

# Your Solution:
There are $6^3$ different ways to roll the three dice, baring in mind we wish to keep the dice ordered based on when they are thrown.

We consider the first die. If the first die rolls a $5$ or $6$, we cannot have strictly increasing order for the two remaining dice.

If the first die rolls a $1$, there are $10$ successful outcomes: $\{ (2,3), (2,4), (2,5), (2,6), (3,4), (3,5), (3,6), (4,5), (4,6), (5,6) \}$

If the first die rolls a $2$, there are $6$ successful outcomes: $\{ (3,4), (3,5), (3,6), (4,5), (4,6), (5,6) \}$

If the first die rolls a $3$, there are $3$ successful outcomes: $\{ (4,5), (4,6), (5,6) \}$

If the first die rolls a $4$, there is $1$ successful outcome:    $\{ (5,6) \}$

This gives us a total of $20$ outcomes that occur in strictly increasing order, thus the probability is $\frac{20}{216}=.092593$



---
## Q2.2: Design (10pt)
Design a Monte Carlo Simulation using numpy and scipy to verify your solution.


In [11]:
import random

def dice_roll():
    die_1 = random.randint(1,6)
    die_2 = random.randint(1,6)
    die_3 = random.randint(1,6)
    if die_1 < die_2 and die_2 < die_3:
        return 1
    else:
        return 0

total_sims = 100000000
#number of simulations
successes = 0
#counts number of successful rolls
iterator = 0

while iterator < total_sims:
    iterator += 1
    if dice_roll() == 1:
        successes += 1
    else:
        continue

probability = successes/total_sims

print("Monte Carlo Simulation of the above inquiry estimates the probability to be:" , probability)

Monte Carlo Simulation of the above inquiry estimates the probability to be: 0.09262867




---



---


# Q3: Gaussian Distribution (30pt)


---

## Q3.1: Moments (15pt)
If $\m{X}$ follows standard normal distribution ( $\m{X} \sim N(0, 1)$ ), what is $\mb{E}[\m{X}^n]$ for $n = 1, 2, 3$
and 4?


# Your Solution:

Firstly, note that for a normal distribution, we have:

$f(x) = \frac{1}{\sqrt{2\pi\sigma^2}}exp[-\frac{(x-\mu)^2}{2\sigma^2}] = \frac{e^{-\frac{x^2}{2}}}{\sqrt{2\pi}}$

The first moment of a normal distribution is the mean, which here is 0, but we still note the integral that gives 0. We also derive the second moment, which we know is $\sigma^2=1$. 

\begin{align*}
\mathbb{E}[X^1] &= \int_{-\infty}^{\infty} x f(x) dx\\
&= \int_{-\infty}^{\infty} x(\frac{e^{-\frac{x^2}{2}}}{\sqrt{2\pi}}) dx\\
&= \frac{1}{\sqrt{2\pi}}\int_{-\infty}^{\infty} xe^{-\frac{x^2}{2}} dx\\
&= 0.\\
\\
\mathbb{E}[X^2] &= \int_{-\infty}^{\infty} x^2 f(x) dx\\
&= \int_{-\infty}^{\infty} x^2(\frac{e^{-\frac{x^2}{2}}}{\sqrt{2\pi}}) dx\\
&= \frac{1}{\sqrt{2\pi}}\int_{-\infty}^{\infty} x^2e^{-\frac{x^2}{2}} dx\\
\text{ Let } &dv = -xe^{-\frac{x^2}{2}} dx,\text{  } v = e^{-\frac{x^2}{2}},\text{  } u = -x,\text{  } du = -1 dx\\
\text{ Then } ... &= \frac{1}{\sqrt{2\pi}}(-xe^{-\frac{x^2}{2}}\Big|_{-\infty}^\infty) - \frac{1}{\sqrt{2\pi}}\int_{-\infty}^{\infty} -e^{-\frac{x^2}{2}} dx\\
&= \frac{1}{\sqrt{2\pi}}\int_{-\infty}^{\infty} e^{-\frac{x^2}{2}} dx\\
\text{ Let } z &= \frac{x}{\sqrt{2}},\text{  } dz = \frac{dx}{\sqrt{2}}\\
\text{ Then } ... &= \frac{1}{\sqrt{2\pi}}\int_{-\infty}^{\infty} \sqrt{2} e^{-z^2} dz\\
&= \frac{1}{\sqrt{\pi}} \int_{-\infty}^{\infty} e^{-z^2}dz\\
&= \frac{1}{\sqrt{\pi}} * \sqrt{\pi}\\
&= 1.\\
\\
\mathbb{E}[X^3] &= \int_{-\infty}^{\infty} x^3 f(x) dx\\
&= \int_{-\infty}^{\infty} x^3(\frac{e^{-\frac{x^2}{2}}}{\sqrt{2\pi}}) dx\\
&= \frac{1}{\sqrt{2\pi}}\int_{-\infty}^{\infty} x^3e^{-\frac{x^2}{2}} dx\\
\text{ Let } &dv = -xe^{-\frac{x^2}{2}} dx,\text{  } v = e^{-\frac{x^2}{2}},\text{  } u = -x^2,\text{  } du = -2x dx\\
\text{ Then } ... &= \frac{1}{\sqrt{2\pi}}(-x^2e^{-\frac{x^2}{2}}\Big|_{-\infty}^\infty) - \frac{1}{\sqrt{2\pi}}\int_{-\infty}^{\infty} -2xe^{-\frac{x^2}{2}} dx\\
&= 2*\frac{1}{\sqrt{2\pi}}\int_{-\infty}^{\infty} xe^{-\frac{x^2}{2}} dx\\
&= 2*\mathbb{E}[X^1]\\
&= 0.\\
\\
\mathbb{E}[X^4] &= \int_{-\infty}^{\infty} x^4 f(x) dx\\
&= \int_{-\infty}^{\infty} x^4(\frac{e^{-\frac{x^2}{2}}}{\sqrt{2\pi}}) dx\\
&= \frac{1}{\sqrt{2\pi}}\int_{-\infty}^{\infty} x^4e^{-\frac{x^2}{2}} dx\\
\text{ Let } &dv = -xe^{-\frac{x^2}{2}} dx,\text{  } v = e^{-\frac{x^2}{2}},\text{  } u = -x^3,\text{  } du = -3x^2 dx\\
\text{ Then } ... &= \frac{1}{\sqrt{2\pi}}(-x^3e^{-\frac{x^2}{2}}\Big|_{-\infty}^\infty) - \frac{1}{\sqrt{2\pi}}\int_{-\infty}^{\infty} -3x^2e^{-\frac{x^2}{2}} dx\\
&= 3*\frac{1}{\sqrt{2\pi}}\int_{-\infty}^{\infty} x^2e^{-\frac{x^2}{2}} dx\\
&= 3*\mathbb{E}[X^2]\\
&= 3.
\end{align*}





---


## Q3.2: Posterior of the mean of a Gaussian with known variance (15pt)
Given i.i.d. data $D= \{x^{(i)}\}_{i=1}^N$, compute $p(\mu| D, \sigma^2)$ for the Bayesian model,
\begin{align}
p(x|\mu) = \frac{1}{\sqrt{2\pi\sigma^2}}\exp\left[-\frac{(x-\mu)^2}{2\sigma^2}\right]
\end{align}
with the prior distribution
\begin{align}
p(\mu; \mu_0, \sigma_0^2)= \frac{1}{\sqrt{2\pi\sigma_0^2}}\exp\left[-\frac{(\mu-\mu_0)^2}{2\sigma_0^2}\right]
\end{align}
where $\sigma^2$ is a fixed known quantity.

**Hint**: You may use that
\begin{align}
N(x; m_1, \sigma_1^2) N(x; m_2, \sigma_2^2) \propto N(x; m_3, \sigma_3^2)
\end{align}
where $$\sigma_3^2 =\left(\frac{1}{\sigma^2_1}+\frac{1}{\sigma^2_2}\right)^{-1} $$ and
$$ m_3 = \sigma^2_3\left(\frac{m_1}{\sigma_1^2}+\frac{m_2}{\sigma_2^2}\right)$$



# Your Solution:

By Bayes Rule, we know 
$$
\begin{align*}
p(\mu| D, \sigma^2) &= p(\mu| D) \\
&= \frac{p(D | \mu)p(\mu)}{\{p(D)\}} \\
&\propto p(D | \mu)p(\mu)
\end{align*}
$$ 

We have (based on the prior distribution) $p(\mu) = \frac{1}{\sqrt{2\pi\sigma_0^2}}\exp\left[-\frac{(\mu-\mu_0)^2}{2\sigma_0^2}\right]$. We can derive $p(D | \mu)$ by the likelyhood, i.e. 
$$
\begin{align*}
\mathcal{L}(D | \mu) &= \prod_{i=0}^{n-1} p(X_i|\mu)\\
&= \prod_{i=0}^{n-1} \frac{1}{\sqrt{2\pi\sigma^2}}\exp\left[-\frac{(x_i-\mu)^2}{2\sigma^2}\right]\\
&= (\frac{1}{\sqrt{2\pi\sigma^2}})^n \exp\left[-\frac{1}{2\sigma^2}\sum_{i=0}^{n-1} (x_i-\mu)^2 \right]\\
\end{align*}
$$

So now we get:
$$
\begin{align*}
p(\mu| D, \sigma^2) &\propto p(D | \mu)p(\mu)\\
&= \frac{1}{\sqrt{2\pi\sigma_0^2}}\exp\left[-\frac{(\mu-\mu_0)^2}{2\sigma_0^2}\right](\frac{1}{\sqrt{2\pi\sigma^2}})^n \exp\left[-\frac{1}{2\sigma^2}\sum_{i=0}^{n-1} (x_i-\mu)^2 \right]\\
&= \{\frac{1}{\sqrt{2\pi\sigma_0^2}}(\frac{1}{\sqrt{2\pi\sigma^2}})^n\}\exp\left[-\frac{(\mu-\mu_0)^2}{2\sigma_0^2}\right]\exp\left[-\frac{1}{2\sigma^2}\sum_{i=0}^{n-1} (x_i-\mu)^2 \right]\\
&\propto \exp\left[-\frac{(\mu-\mu_0)^2}{2\sigma_0^2}-\frac{1}{2\sigma^2}\sum_{i=0}^{n-1} (x_i-\mu)^2 \right]\\
&= \exp\left[-\frac{(\mu^2 - 2\mu\mu_0 + \mu_0^2)}{2\sigma_0^2}-\frac{1}{2\sigma^2}\sum_{i=0}^{n-1} (x_i^2 - 2x_i\mu + \mu^2) \right]\\
&= \exp\left[-\frac{\mu^2}{2\sigma_0^2} + \frac{2\mu\mu_0}{2\sigma_0^2} + \{-\frac{\mu_0^2}{2\sigma_0^2} -\frac{1}{2\sigma^2}\sum_{i=0}^{n-1}x_i^2\} + \frac{1}{2\sigma^2} \sum_{i=0}^{n-1} 2x_i\mu - \frac{n\mu^2}{2\sigma^2} \right]\\
&\propto \exp\left[-\frac{\mu^2}{2\sigma_0^2} + \frac{2\mu\mu_0}{2\sigma_0^2} + \frac{1}{2\sigma^2} \sum_{i=0}^{n-1} 2x_i\mu - \frac{n\mu^2}{2\sigma^2} \right]\\
&= \exp\left[-\frac{\mu^2\sigma^2}{2\sigma^2\sigma_0^2} - \frac{n\mu^2\sigma_0^2}{2\sigma^2\sigma_0^2} + \frac{2\mu\mu_0\sigma^2}{2\sigma^2\sigma_0^2} + \frac{2\mu\sigma_0^2}{2\sigma^2\sigma_0^2} \sum_{i=0}^{n-1} x_i \right]\\
&= \exp\left[-\frac{1}{2\sigma^2\sigma_0^2} [ (\sigma^2 + n\sigma_0^2)\mu^2 - 2(\mu_0\sigma^2 + \sigma_0^2 \sum_{i=0}^{n-1} x_i )\mu ] \right]\\
\text{Using Completing the Square method, we can see that}\\
&= \exp\left[-\frac{1}{2\sigma^2\sigma_0^2} [ a(\mu - d)^2 + e ] \right]\\
\text{where for a polynomial of the form } au^2+bu+c,&\\
a=(\sigma^2 + n\sigma_0^2),&\\ 
b=-2(\mu_0\sigma^2 + \sigma_0^2 \sum_{i=0}^{n-1} x_i ),& \\
c=0,&\\
\text{ and } d=-\frac{b}{2a}=\frac{2(\mu_0\sigma^2 + \sigma_0^2 \sum_{i=0}^{n-1} x_i)}{2(\sigma^2 + n\sigma_0^2)},&\\
e = c-\frac{b^2}{4a} = -\frac{4(\mu_0\sigma^2 + \sigma_0^2 \sum_{i=0}^{n-1} x_i)^2}{4(\sigma^2 + n\sigma_0^2)}.&\\
\text{So, } p(\mu | D) &= \exp\left[-\frac{1}{2\sigma^2\sigma_0^2} [ (\sigma^2 + n\sigma_0^2)(\mu-\frac{\mu_0\sigma^2 + \sigma_0^2 \sum_{i=0}^{n-1} x_i}{\sigma^2 + n\sigma_0^2})^2 - \{\frac{(\mu_0\sigma^2 + \sigma_0^2 \sum_{i=0}^{n-1} x_i)^2}{\sigma^2 + n\sigma_0^2} \} ]\right]\\
&\propto \exp\left[-\frac{\sigma^2 + n\sigma_0^2}{2\sigma^2\sigma_0^2} (\mu-\frac{\mu_0\sigma^2 + \sigma_0^2 \sum_{i=0}^{n-1} x_i}{\sigma^2 + n\sigma_0^2})^2\right].\\
\text{Thus, the probability of } \mu \text{ given the data } D \text{, a variance } \sigma^2 \text{,}&\\
\text{and the prior distribution is proportional to a normal distribution with:}&\\
\text{mean} &= \frac{\mu_0\sigma^2 + \sigma_0^2 \sum_{i=0}^{n-1} x_i}{\sigma^2 + n\sigma_0^2}\\
\text{and variance}&= \frac{\sigma^2\sigma_0^2}{\sigma^2 + n\sigma_0^2}
\end{align*} 
$$



---



---


# Q4: Markov Chain (30pt)
## Q4.1 (10pt)
Let $X$ be a simple random walk (i.e., $X$ increases by one with probability $p$ and decreases by 1 with probability $q$) and $Y_n=X_{2n}$. Compute the transition matrix for $Y$.

# Your Solution:

If we assume we are at step $i$ at the start, then we have a probability $p$ to go to step $i+1$ and a probability $q$ to go to step $i-1$ with no probability to go anywhere else, so then the $i^{th}$ row of $P^X$ is $P_i^X = [ ... , q, 0, p, ...  ]$ where the $...$ indicates that the rest of the vector is 0's. Since Y is simply taking two steps of X,  we have a probability $p^2$ to go to 2 steps forward, i.e. $i+2$, a probability $q^2$ to go 2 steps backward, i.e. $i-2$ and a probability of $pq$ to go either one step forward then one step back, or a probability of $qp$ to go either one step back then one step forward, where we stay at step $i$ for either, thus the probability of staying at step $i$ is $2pq$. So the $i^{th}$ row of $P^Y$ is $P_i^Y = [ ... q^2, 0, 2pq, 0, p^2, ...  ]$, where the $...$ indicates that the rest of the vector is 0's. This does indeed form a Markov Transition Matrix since $p+q=1$, so for each row of $P^Y$, we have $p^2+2pq+q^2 = (p+q)^2 = 1^2 = 1$. Thus, $P^Y$ (centered at the $i^{th}$ row and column) is:

$$
\begin{bmatrix}
...\\
...&q^2&0&2pq&0&p^2&0&0&0&0&...\\ 
...&0&q^2&0&2pq&0&p^2&0&0&0&...\\
...&0&0&q^2&0&2pq&0&p^2&0&0&...\\
...&0&0&0&q^2&0&2pq&0&p^2&0&...\\
...&0&0&0&0&q^2&0&2pq&0&p^2&...\\
...\\
\end{bmatrix}$$








---


# Q4.2: Which of the following is a Markov chain (20pt)

A six-sided die is rolled repeatedly. Which of the following a Markov chain? For those
that are, find the transition matrix.

- $X_n$ is the largest number rolled up to the $n$th roll.

- $X_n$ is the number of sixes rolled in the first $n$ rolls.

- At time $n$, $X_n$ is the time since the last six was rolled.

- At time $n$, $X_n$ is the time until the next six is rolled.


# Your Solution:
- $X_n$ is the largest number rolled up to the $n$th roll. This is a Markov Chain (assuming we know what $X_1$ is) with the following transition matrix:

$$
\begin{bmatrix}
\frac{1}{6}&\frac{1}{6}&\frac{1}{6}&\frac{1}{6}&\frac{1}{6}&\frac{1}{6}\\
0&\frac{2}{6}&\frac{1}{6}&\frac{1}{6}&\frac{1}{6}&\frac{1}{6}\\
0&0&\frac{3}{6}&\frac{1}{6}&\frac{1}{6}&\frac{1}{6}\\
0&0&0&\frac{4}{6}&\frac{1}{6}&\frac{1}{6}\\
0&0&0&0&\frac{5}{6}&\frac{1}{6}\\
0&0&0&0&0&1\\
\end{bmatrix}
$$

- $X_n$ is the number of sixes rolled in the first $n$ rolls. This is a Markov Chain with the following transition matrix:

$$
\begin{bmatrix}
\frac{5}{6}&\frac{1}{6}&0&0&0&...\\
0&\frac{5}{6}&\frac{1}{6}&0&0&...\\
0&0&\frac{5}{6}&\frac{1}{6}&0&...\\
0&0&0&\frac{5}{6}&\frac{1}{6}&...\\
...
\end{bmatrix}
$$

- At time $n$, $X_n$ is the time since the last six was rolled. This is a Markov Chain with the following transition matrix:

$$
\begin{bmatrix}
\frac{1}{6}&\frac{5}{6}&0&0&0&...\\
\frac{1}{6}&0&\frac{5}{6}&0&0&...\\
\frac{1}{6}&0&0&\frac{5}{6}&0&...\\
\frac{1}{6}&0&0&0&\frac{5}{6}&...\\
...
\end{bmatrix}
$$

- At time $n$, $X_n$ is the time until the next six is rolled. This is not a Markov Chain since we need to know when the next six is rolled to know what $X_n$ is, i.e. $X_n$ is based on some $X_{n+m}$ for some $m\geq1$.